# Collecting data from html files

### Importing necessary paackages, function

In [3]:
from datacollection import html_to_df
import pandas as pd
from datetime import datetime
import time
from timeit import default_timer as timer
from multiprocessing import Pool

### Defining paths to execute function

In [22]:
path1 = 'G:/python_project/Resources/Osoby_HU_html/7'
path2 = 'G:/python_project/Resources/Osoby_HU_html/8'
path3 = 'G:/python_project/Resources/Osoby_HU_html/9'
path4 = 'G:/python_project/Resources/Osoby_HU_html/10'

In [23]:
#test paths
#path5 ='C:/Dokumentumok/Egyetem/Heller/Kurzusok/Haladó_Python/projekt/html'
#path6 ='C:/Dokumentumok/Egyetem/Heller/Kurzusok/Haladó_Python/projekt/html2'
#path7 ='C:/Dokumentumok/Egyetem/Heller/Kurzusok/Haladó_Python/projekt/html3'
#path8 ='C:/Dokumentumok/Egyetem/Heller/Kurzusok/Haladó_Python/projekt/html4'

### Creating multiple processes to read data from folders

In [24]:
if __name__ == '__main__':
    
    start = timer()
    
    with Pool() as pool:
        df = pd.concat(pool.map(html_to_df, [path1, path2, path3, path4]))
    
    end = timer()
    print(f'elapsed time: {end - start}')

UnboundLocalError: local variable 'column_list' referenced before assignment

In [12]:
df

,id,first_name_1,first_name_2,last_name,birth_date,birth_place,death_date,death_place,father_id,mother_id,partner_id1,partner_id2,partner_id3,partner_id4,partner_id5,partner_id6
0,I740001KOM,Juraj,,Šarkan,1746-04-27,Tótkomlós,1746-05-25,Tótkomlós,i100001000,,NaN,NaN,NaN,NaN,NaN,NaN
1,I740002KOM,Pavol,,Tót,1746-05-25,Tótkomlós,,,,,NaN,NaN,NaN,NaN,NaN,NaN
2,I740003KOM,Ján,,Lomský,1746-05-31,Tótkomlós,,,,,NaN,NaN,NaN,NaN,NaN,NaN
3,I740004KOM,Pavol,,Dudík,1746-06-12,Tótkomlós,,,i100025000,i100024000,NaN,NaN,NaN,NaN,NaN,NaN
4,I740005KOM,Mária,,Tuška,1746-06-19,Tótkomlós,1793-03-04,Tótkomlós,i100027000,i100026000,i100592000,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5154,i102971326,Ondrej,,Mokoš,1794-01-01,,1846-07-26,Nadlak,,,i102971325,,,,,
5155,i190000001,Ján,,Kováč,1899-01-01,Kaszaper,1899-11-27,Kaszaper,,,NaN,NaN,NaN,NaN,NaN,NaN
5156,i190000002,Mária,,Rác,1897-01-01,Végegyháza,1899-04-22,Tótbánhegyes,,,NaN,NaN,NaN,NaN,NaN,NaN
5157,i190000003,Juraj,,Sýkora,1858-01-01,Medgyesbodzás,1920-04-10,Nagybánhegyes,,,I860095BAN,,,,,


### Creating csv

In [15]:
df.to_csv("final.csv", encoding='utf-8-sig', sep = ';', index = False)

## Reading csv and cleaning the data 

In [4]:
df_new = pd.read_csv('final.csv', sep = ';')

C:\Users\pipis\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df_new.replace({"*" : None}, inplace = True)
df_new.replace({"-" : None}, inplace = True)
df_new.replace({"+" : None}, inplace = True)
df_new.replace({"?" : None}, inplace = True)
df_new.replace({"---" : None}, inplace = True)
df_new.replace({"(+)" : None}, inplace = True)


## Adding gender column


#### Creating lists of id-s that are males and females by fater_id, mother_id columns

In [6]:
s1 = df_new[["father_id"]]
s1.columns = ["id"]
s2 = df_new[["mother_id"]]
s2.columns = ["id"]
males_list = s1.drop_duplicates()["id"].tolist()
females_list = s2.drop_duplicates()["id"].tolist()

#### Creating list of first names of males and females

In [7]:
male_names_list = df_new[df_new["id"].isin(males_list)]["first_name_1"].drop_duplicates().tolist()
female_names_list = df_new[df_new["id"].isin(females_list)]["first_name_1"].drop_duplicates().tolist()
print(len(male_names_list))
print(len(female_names_list))

57
50


#### Checking how many names are not in the lists of father or mother names

In [8]:
first_names = df_new["first_name_1"].drop_duplicates().tolist()
missing = [x for x in first_names if ((x not in male_names_list) and (x not in female_names_list))]
len(missing)

76

In [9]:
# Using that in Slovakian almost all of the female names end with a,
# I am creating a list with a endings and not a endings

a = [x for x in missing if not pd.isna(x) and x[-1] == 'a']
nota = [x for x in missing if not pd.isna(x) and x[-1] != 'a']
print(a)
print(nota)

['Apolónia', 'Alojzia', 'Rozina', 'Bria', 'Agneša', 'Savita', 'Emerika', 'Matilda', 'Klementína', 'Antónia', 'Jozefína', 'Justína', 'Eugénia', 'Regina', 'Lívia', 'Lujza', 'Hermína', 'Ilma', 'Otília', 'Zlatica', 'Priska', 'Šarlota', 'Kornélia', 'Dalma', 'Izabela', 'Edita', 'zuzana', 'Henrieta', 'Želmíra', 'Blanka', 'Monika', 'Viola', 'Eleonóra', 'Čilla', 'Perla', 'Erika', 'Hortenzia', 'Xénia', 'Oľga', 'Augusta', 'Marianna', 'Hedviga', 'Mariana', 'Olga', 'Jarmila']
['Demeter', 'Kristián', 'Vavrinec', 'Ferdinand', 'Albert', 'Adolf', 'Abrahám', 'Edmund', 'Eduard', 'Gustáv', 'Alojz', 'Emil', 'Arpád', 'Ákoš', 'Dávid', 'Tibor', 'Florián', 'Sáli', 'Svetozár', 'Miloslav', 'Miloš', 'Rudolf', 'Jafet', 'Viktor', 'Ábel', 'Alfonz', 'Dušan', 'Diamand', 'Čongor', 'Miroslav']


In [10]:
# While sense checking the data I found 3 names that I am not sure
# what gender they belong to. Filtering for these.

df_new[df_new["first_name_1"].isin(["Sáli", "Diamand", "Jafet"]) ]

,id,first_name_1,first_name_2,last_name,birth_date,birth_place,death_date,death_place,father_id,mother_id,partner_id1,partner_id2,partner_id3,partner_id4,partner_id5,partner_id6
46753,I891041KOM,Sáli,Mária,Bočkai,1892-08-24,Tótkomlós,1892-12-06,Tótkomlós,I851063KOM,I850122KOM,NaN,NaN,NaN,NaN,NaN,NaN
55502,I911643KOM,Jafet,NaN,Chlebnický,1914-06-02,Tótkomlós,1915-07-19,Tótkomlós,I861149KOM,I870796KOM,NaN,NaN,NaN,NaN,NaN,NaN
63132,I941279KOM,Diamand,Gabriela,Domanovský,NaN,NaN,NaN,NaN,i091604000,I920314KOM,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Second first names tells that the first two names are 
# female names. The last is unkonwn so it'll remain male.

# Reorganising the two lists

a.extend(["Sáli", "Diamand"])
nota.remove("Sáli")
nota.remove("Diamand") 

#### Creating dictionary with names and gender

In [12]:
d = dict(zip(male_names_list, ["male"] * len(male_names_list)))
d.update(dict(zip(nota, ["male"] * len(nota))))
d.update(dict(zip(female_names_list, ["female"] * len(female_names_list))))
d.update(dict(zip(a, ["female"] * len(a))))

#### Adding gender column to original dataframe

In [13]:
df_new["sex"] = df_new["first_name_1"].map(d)
df_new

,id,first_name_1,first_name_2,last_name,birth_date,birth_place,death_date,death_place,father_id,mother_id,partner_id1,partner_id2,partner_id3,partner_id4,partner_id5,partner_id6,sex
0,I740001KOM,Juraj,NaN,Šarkan,1746-04-27,Tótkomlós,1746-05-25,Tótkomlós,i100001000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,male
1,I740002KOM,Pavol,NaN,Tót,1746-05-25,Tótkomlós,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,male
2,I740003KOM,Ján,NaN,Lomský,1746-05-31,Tótkomlós,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,male
3,I740004KOM,Pavol,NaN,Dudík,1746-06-12,Tótkomlós,NaN,NaN,i100025000,i100024000,NaN,NaN,NaN,NaN,NaN,NaN,male
4,I740005KOM,Mária,NaN,Tuška,1746-06-19,Tótkomlós,1793-03-04,Tótkomlós,i100027000,i100026000,i100592000,NaN,NaN,NaN,NaN,NaN,female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68659,i102971326,Ondrej,NaN,Mokoš,1794-01-01,NaN,1846-07-26,Nadlak,NaN,NaN,i102971325,NaN,NaN,NaN,NaN,NaN,male
68660,i190000001,Ján,NaN,Kováč,1899-01-01,Kaszaper,1899-11-27,Kaszaper,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,male
68661,i190000002,Mária,NaN,Rác,1897-01-01,Végegyháza,1899-04-22,Tótbánhegyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,female
68662,i190000003,Juraj,NaN,Sýkora,1858-01-01,Medgyesbodzás,1920-04-10,Nagybánhegyes,NaN,NaN,I860095BAN,NaN,NaN,NaN,NaN,NaN,male


## Writing new csv file

In [14]:
df_new.to_csv("final_cleaned.csv", encoding='utf-8-sig', sep = ';', index = False)